# Succinate production hands-on session

This notebook demonstrates how to load the yeast9 genome-scale metabolic model,
configure the extracellular medium, and explore environmental conditions that
maximize succinate secretion using flux balance analysis (FBA).


## 1. Environment preparation

The Python utilities shipped with the repository expect a `.env` file in the
project root and a working COBRApy installation. Create and activate a virtual
environment, install the dependencies (including plotting helpers), and ensure
that the working directory of this notebook is the repository root.

```bash
python -m venv .venv
source .venv/bin/activate
pip install "cobra[glpk]" python-dotenv pandas matplotlib seaborn
touch .env
```

Once the prerequisites are installed, the remaining cells can be executed in
sequence.

In [ ]:
from pathlib import Path

# Ensure that the repository root contains the marker file required by code.io
Path(".env").touch()
print('`.env` file ready for use.')


In [ ]:
import itertools
from typing import Dict, Iterable
import warnings

import cobra
import pandas as pd
from cobra.flux_analysis import pfba
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from code.io import read_yeast_model

sns.set_theme(style='whitegrid')
# Suppress repeated infeasibility warnings during grid searches; remove this
# filter if you want to inspect every solver status message.
warnings.filterwarnings(
    'ignore',
    message="Solver status is 'infeasible'.",
    category=UserWarning,
    module='cobra.util.solver',
)


## 2. Load the yeast9 model

The helper `read_yeast_model` locates the SBML file shipped with the repository
and returns a COBRApy `Model` instance.


In [ ]:
model = read_yeast_model()

print(f'Reactions: {len(model.reactions)}')
print(f'Metabolites: {len(model.metabolites)}')
print(f'Genes: {len(model.genes)}')
print(f'Default objective: {model.objective.expression}')


## 3. Configure a base medium

We reproduce the Kennedy synthetic complete medium used in the MATLAB utilities
(`complete_Y7`). All exchange reactions are first reset to prohibit uptake, then
selected nutrients are reopened with specific uptake bounds (negative flux
corresponds to import).


In [ ]:
# Exchange reactions that receive a constrained uptake of -0.5 mmol gDW^-1 h^-1
CONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1604', 'r_1639', 'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1671',
    'r_1883', 'r_1757', 'r_1891', 'r_1889', 'r_1810', 'r_1993', 'r_1893',
    'r_1897', 'r_1947', 'r_1899', 'r_1900', 'r_1902', 'r_1967', 'r_1903',
    'r_1548', 'r_1904', 'r_2028', 'r_2038', 'r_1906', 'r_2067', 'r_1911',
    'r_1912', 'r_1913', 'r_2090', 'r_1914', 'r_2106',
)

# Exchange reactions that remain unconstrained for uptake (-1000 lower bound)
UNCONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1672',  # ammonium
    'r_1654',  # potassium
    'r_1992',  # oxygen
    'r_2005',  # phosphate
    'r_2060',  # sulfate
    'r_1861',  # iron
    'r_1832',  # proton
    'r_2100',  # water
    'r_4593',  # chloride
    'r_4595',  # manganese
    'r_4596',  # zinc
    'r_4597',  # magnesium
    'r_2049',  # sodium
    'r_4594',  # copper
    'r_4600',  # calcium
    'r_2020',  # sulfate/glutathione coupled import
)

GLUCOSE_EXCHANGE_ID = 'r_1714'
SUCCINATE_EXCHANGE_ID = 'r_2056'
BIOMASS_REACTION_ID = 'r_2111'
CITRATE_EXCHANGE_ID = 'r_1687'
GLUTAMATE_EXCHANGE_ID = 'r_1889'
MALATE_EXCHANGE_ID = 'r_1552'
LACTATE_EXCHANGE_ID = 'r_1551'
FORMATE_EXCHANGE_ID = 'r_1793'
ACETATE_EXCHANGE_ID = 'r_1634'

ORGANIC_ACID_EXCHANGES: Dict[str, str] = {
    'succinate_flux': SUCCINATE_EXCHANGE_ID,
    'citrate_flux': CITRATE_EXCHANGE_ID,
    'malate_flux': MALATE_EXCHANGE_ID,
    'lactate_flux': LACTATE_EXCHANGE_ID,
    'formate_flux': FORMATE_EXCHANGE_ID,
    'acetate_flux': ACETATE_EXCHANGE_ID,
    'glutamate_flux': GLUTAMATE_EXCHANGE_ID,
}

ORGANIC_ACID_LABELS: Dict[str, str] = {
    'succinate_flux': 'Succinate',
    'citrate_flux': 'Citric acid',
    'malate_flux': 'Malic acid',
    'lactate_flux': 'Lactic acid',
    'formate_flux': 'Formic acid',
    'acetate_flux': 'Acetic acid',
    'glutamate_flux': 'Glutamic acid',
}

ORGANIC_ACID_COLUMNS = list(ORGANIC_ACID_EXCHANGES.keys())

# Feed regimes with illustrative conversion from percentage supplement to uptake bound.
# Adjust the lower bounds to match your experimental feed rates.
FEED_REGIMES: Dict[str, Dict[str, float]] = {
    'none': {},
    'citrate_1pct': {CITRATE_EXCHANGE_ID: -5.0},
    'citrate_3pct': {CITRATE_EXCHANGE_ID: -15.0},
    'glutamate_0_5pct': {GLUTAMATE_EXCHANGE_ID: -2.5},
    'glutamate_1pct': {GLUTAMATE_EXCHANGE_ID: -5.0},
}

FEED_LABELS: Dict[str, str] = {
    'none': 'No supplement',
    'citrate_1pct': 'Citrate 1%',
    'citrate_3pct': 'Citrate 3%',
    'glutamate_0_5pct': 'Glutamate 0.5%',
    'glutamate_1pct': 'Glutamate 1%',
}


def apply_kennedy_medium(model: cobra.Model, glucose_limit: float = -20.0) -> cobra.Model:
    """Reset exchange bounds and apply the Kennedy synthetic complete medium."""
    for rxn in model.exchanges:
        rxn.lower_bound = 0.0
        rxn.upper_bound = 1000.0
    for rxn_id in CONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -0.5
    model.reactions.get_by_id(GLUCOSE_EXCHANGE_ID).lower_bound = glucose_limit
    for rxn_id in UNCONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -1000.0
    return model


def apply_feed_regime(model: cobra.Model, feed_regime: str) -> None:
    """Override exchange lower bounds for optional feed supplements."""
    overrides = FEED_REGIMES.get(feed_regime, {})
    for rxn_id, lower_bound in overrides.items():
        model.reactions.get_by_id(rxn_id).lower_bound = lower_bound

In [ ]:
baseline_model = read_yeast_model()
apply_kennedy_medium(baseline_model)

succinate_exchange = baseline_model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
baseline_model.objective = succinate_exchange
baseline_solution = baseline_model.optimize()

print(f'Baseline succinate flux: {baseline_solution.objective_value:.4f} mmol gDW^-1 h^-1')
print(f'Associated biomass flux: {baseline_solution.fluxes.get(BIOMASS_REACTION_ID, float("nan")):.4f}')
print(f'Optimization status: {baseline_solution.status}')


## 4. Explore environmental scenarios

We now parameterize a helper that reapplies the base medium, tweaks a set of environmental constraints (glucose feed, oxygen availability, proton uptake as a proxy for extracellular pH), toggles optional citrate/glutamate supplementation, and maximizes succinate secretion. Alongside the succinate objective, the workflow records the secretion rates of citric, malic, lactic, formic, acetic, and glutamic acids so you can compare how each metabolite responds to the tested conditions.

In [ ]:
def evaluate_environment(
    model: cobra.Model,
    glucose_limit: float,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float = -10.0,
    temperature_tag: str = '30C',
    feed_regime: str = 'none',
) -> Dict[str, float]:
    """Solve an FBA problem that maximizes succinate secretion."""
    with model:
        apply_kennedy_medium(model, glucose_limit=glucose_limit)
        model.reactions.get_by_id('r_1992').lower_bound = oxygen_limit
        model.reactions.get_by_id('r_1832').lower_bound = proton_limit
        model.reactions.get_by_id('r_1672').lower_bound = ammonium_limit
        apply_feed_regime(model, feed_regime)

        # Optionally adjust non-growth-associated maintenance to mimic temperature shifts
        if temperature_tag == '37C':
            model.reactions.get_by_id('r_4041').lower_bound = 10.0
        elif temperature_tag == '20C':
            model.reactions.get_by_id('r_4041').lower_bound = 6.0

        model.objective = model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        solution = model.optimize()

        acid_fluxes = {key: float('nan') for key in ORGANIC_ACID_EXCHANGES}
        biomass_flux = float('nan')
        if solution.status == 'optimal':
            acid_fluxes = {
                flux_column: solution.fluxes.get(rxn_id, float('nan'))
                for flux_column, rxn_id in ORGANIC_ACID_EXCHANGES.items()
            }
            biomass_flux = solution.fluxes.get(BIOMASS_REACTION_ID, float('nan'))
        return {
            'status': solution.status,
            **acid_fluxes,
            'biomass_flux': biomass_flux,
            'glucose_limit': glucose_limit,
            'oxygen_limit': oxygen_limit,
            'proton_limit': proton_limit,
            'ammonium_limit': ammonium_limit,
            'temperature_tag': temperature_tag,
            'feed_regime': feed_regime,
        }

In [ ]:
analysis_model = read_yeast_model()
apply_kennedy_medium(analysis_model)

glucose_grid = (-10.0, -15.0, -20.0)
oxygen_grid = (0.0, -5.0, -10.0)
proton_grid = (-1.0, -5.0, -10.0)
temperature_grid = ('20C', '30C', '37C')
feed_regimes = ('none', 'citrate_1pct', 'citrate_3pct', 'glutamate_0_5pct', 'glutamate_1pct')

records = []
for glucose_limit, oxygen_limit, proton_limit, temperature_tag, feed_regime in itertools.product(
    glucose_grid, oxygen_grid, proton_grid, temperature_grid, feed_regimes
):
    record = evaluate_environment(
        analysis_model,
        glucose_limit=glucose_limit,
        oxygen_limit=oxygen_limit,
        proton_limit=proton_limit,
        ammonium_limit=-10.0,
        temperature_tag=temperature_tag,
        feed_regime=feed_regime,
    )
    records.append(record)

results = pd.DataFrame(records)
results['feed_label'] = results['feed_regime'].map(FEED_LABELS)
acid_columns = ORGANIC_ACID_COLUMNS
summary_columns = acid_columns + [
    'biomass_flux',
    'glucose_limit',
    'oxygen_limit',
    'proton_limit',
    'temperature_tag',
    'feed_label',
]
results.sort_values('succinate_flux', ascending=False)[summary_columns].head(10)

Heatmaps make it easier to compare how oxygen and proton limits influence metabolite secretion at each tested temperature. Only feasible scenarios (solver status `optimal`) are shown below. Call `plot_acid_heatmaps` with a different `flux_column` to inspect citric, malic, lactic, formic, acetic, or glutamic acid in addition to succinate.

In [ ]:
def plot_acid_heatmaps(results: pd.DataFrame, flux_column: str = 'succinate_flux', feed_order=None) -> None:
    """Visualize secretion rates across oxygen and proton limits for a metabolite."""
    feasible = results[results["status"] == "optimal"].copy()
    if feasible.empty:
        print("No feasible scenarios to visualize.")
        return
    feed_levels = feed_order or list(dict.fromkeys(feasible["feed_regime"]))
    temperatures = list(dict.fromkeys(feasible["temperature_tag"]))
    n_rows = len(feed_levels)
    n_cols = len(temperatures)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 3.5 * n_rows), sharex=True, sharey=True)
    if n_rows == 1 and n_cols == 1:
        axes = [[axes]]
    elif n_rows == 1:
        axes = [axes]
    elif n_cols == 1:
        axes = [[ax] for ax in axes]
    metabolite_label = ORGANIC_ACID_LABELS.get(flux_column, flux_column)
    for i, feed_regime in enumerate(feed_levels):
        feed_label = FEED_LABELS.get(feed_regime, feed_regime)
        for j, temperature in enumerate(temperatures):
            ax = axes[i][j]
            subset = feasible[(feasible["feed_regime"] == feed_regime) & (feasible["temperature_tag"] == temperature)]
            if subset.empty:
                ax.axis('off')
                ax.set_title(f"Temperature: {temperature}")
                continue
            pivot = subset.pivot_table(
                index="oxygen_limit",
                columns="proton_limit",
                values=flux_column,
                aggfunc="mean",
            )
            sns.heatmap(
                pivot.sort_index(ascending=False),
                annot=True,
                fmt=".2f",
                cmap="viridis",
                ax=ax,
                cbar=(i == 0 and j == n_cols - 1),
                cbar_kws={"shrink": 0.75},
            )
            ax.set_title(f"Temperature: {temperature}")
            if j == 0:
                ax.set_ylabel(
                    "
".join([feed_label, "Oxygen lower bound (mmol gDW$^{-1}$ h$^{-1}$)"])
                )
            else:
                ax.set_ylabel("")
            if i == n_rows - 1:
                ax.set_xlabel("Proton lower bound (mmol gDW$^{-1}$ h$^{-1}$)")
            else:
                ax.set_xlabel("")
    fig.suptitle(f"{metabolite_label} secretion (mmol gDW$^{-1}$ h$^{-1}$)", y=1.02)
    fig.tight_layout()

plot_acid_heatmaps(results)

### Growth-production trade-off

A scatter plot highlights how secretion of the selected metabolite relates to biomass production. Marker colors encode the feed supplement, shapes indicate the temperature set point, and annotations list the glucose, oxygen, and proton bounds for each scenario. Adjust the `flux_column` argument to compare succinate against other organic acids.

In [ ]:
def plot_growth_tradeoff(results: pd.DataFrame, flux_column: str = 'succinate_flux') -> None:
    """Plot secretion against biomass growth for feasible scenarios."""
    feasible = results[results["status"] == "optimal"].copy()
    if feasible.empty:
        print("No feasible scenarios to visualize.")
        return
    feasible["glucose_feed"] = feasible["glucose_limit"].astype(str)
    feasible["feed_label"] = feasible["feed_regime"].map(FEED_LABELS)
    feasible = feasible.sort_values([flux_column, "biomass_flux"], ascending=False)
    metabolite_label = ORGANIC_ACID_LABELS.get(flux_column, flux_column)
    fig, ax = plt.subplots(figsize=(7, 4))
    sns.scatterplot(
        data=feasible,
        x="biomass_flux",
        y=flux_column,
        hue="feed_label",
        style="temperature_tag",
        s=120,
        ax=ax,
    )
    for _, row in feasible.iterrows():
        label = f"Glu {row['glucose_limit']} | O2 {row['oxygen_limit']} | H+ {row['proton_limit']}"
        ax.annotate(
            label,
            (row["biomass_flux"], row[flux_column]),
            textcoords="offset points",
            xytext=(6, 4),
            fontsize=8,
        )
    ax.set_xlabel("Biomass flux (1 h$^{-1}$)")
    ax.set_ylabel(f"{metabolite_label} secretion (mmol gDW$^{-1}$ h$^{-1}$)")
    ax.set_title(f"{metabolite_label} production vs. growth")
    ax.axhline(0, color="black", linewidth=0.8, linestyle="--", alpha=0.7)
    fig.tight_layout()

plot_growth_tradeoff(results)

### Comparing feed supplements

The feed settings below assume that 1% citrate corresponds to a maximum uptake of
5 mmol gDW$^{-1}$ h$^{-1}$ (scaled proportionally for 3%), while 1% and 0.5% glutamate
feeds enable 5 and 2.5 mmol gDW$^{-1}$ h$^{-1}$ uptake respectively. Adjust the
`FEED_REGIMES` table to reflect measured feed rates if your process uses different
conversion factors. The summary table ranks each supplement by the best succinate
flux achieved in the sweep, and the accompanying heatmap contrasts the maximum
secretion rates for every tracked organic acid.

In [ ]:
def summarize_feed_performance(results: pd.DataFrame) -> pd.DataFrame:
    feasible = results[results['status'] == 'optimal'].copy()
    empty_columns = [
        'feed_regime',
        'feed_label',
        'max_succinate',
        'median_succinate',
        'best_biomass',
    ] + [f"max_{col.replace('_flux', '')}" for col in ORGANIC_ACID_COLUMNS if col != 'succinate_flux']
    if feasible.empty:
        return pd.DataFrame(columns=empty_columns)
    aggregation = {col: 'max' for col in ORGANIC_ACID_COLUMNS}
    aggregation['biomass_flux'] = 'max'
    grouped = feasible.groupby('feed_regime').agg(aggregation)
    grouped['median_succinate'] = feasible.groupby('feed_regime')['succinate_flux'].median()
    rename_map = {
        'succinate_flux': 'max_succinate',
        'biomass_flux': 'best_biomass',
    }
    rename_map.update({
        flux_col: f"max_{flux_col.replace('_flux', '')}"
        for flux_col in ORGANIC_ACID_COLUMNS
        if flux_col != 'succinate_flux'
    })
    grouped = grouped.rename(columns=rename_map)
    grouped['feed_label'] = grouped.index.map(FEED_LABELS)
    acid_metric_columns = [f"max_{col.replace('_flux', '')}" for col in ORGANIC_ACID_COLUMNS]
    ordered_columns = [
        'feed_regime',
        'feed_label',
        'max_succinate',
        'median_succinate',
        'best_biomass',
    ] + [col for col in acid_metric_columns if col != 'max_succinate']
    return grouped.reset_index(drop=True)[ordered_columns]


def plot_feed_vs_acid_maxima(results: pd.DataFrame) -> None:
    feasible = results[results['status'] == 'optimal'].copy()
    if feasible.empty:
        print('No feasible scenarios to visualize.')
        return
    feed_levels = [feed for feed in FEED_LABELS if feed in feasible['feed_regime'].unique()]
    summary = (
        feasible.groupby('feed_regime')[ORGANIC_ACID_COLUMNS]
        .max()
        .reindex(feed_levels)
        .rename(columns=ORGANIC_ACID_LABELS)
    )
    summary.index = [FEED_LABELS.get(feed, feed) for feed in summary.index]
    plt.figure(figsize=(1.8 * len(ORGANIC_ACID_COLUMNS), 3.8))
    sns.heatmap(
        summary,
        annot=True,
        fmt='.2f',
        cmap='rocket_r',
        cbar_kws={'shrink': 0.8},
    )
    plt.xlabel('Metabolite')
    plt.ylabel('Feed regime')
    plt.title('Maximum organic acid secretion by feed regime')
    plt.tight_layout()


feed_summary = summarize_feed_performance(results)
feed_summary

if not feed_summary.empty:
    fig, ax = plt.subplots(figsize=(6, 3.5))
    sns.barplot(data=feed_summary, x='feed_label', y='max_succinate', ax=ax, color='#1f77b4')
    ax.set_ylabel('Max succinate flux (mmol gDW$^{-1}$ h$^{-1}$)')
    ax.set_xlabel('Feed regime')
    ax.set_title('Best succinate flux by feed supplement')
    ax.tick_params(axis='x', rotation=20)
    fig.tight_layout()
    plot_feed_vs_acid_maxima(results)

### Succinate-focused scenario report
The table below highlights the *main* succinate-producing scenario for each feed supplement. For every feed regime, we identify the environmental setting that maximizes succinate flux and display the accompanying organic acid secretion rates (*sub results*) observed under the same conditions. Use this table to see how citric, malic, lactic, formic, acetic, and glutamic acid production behaves when succinate is optimized.

In [ ]:
def summarize_succinate_context(results: pd.DataFrame) -> pd.DataFrame:
    feasible = results[results['status'] == 'optimal'].copy()
    if feasible.empty:
        columns = [
            'Feed label', 'Temperature', 'Glucose limit', 'Oxygen limit', 'Proton limit',
            'Ammonium limit', 'Biomass flux',
        ] + [ORGANIC_ACID_LABELS.get(col, col) for col in ORGANIC_ACID_COLUMNS]
        return pd.DataFrame(columns=columns)
    feasible = feasible.copy()
    feasible['feed_label'] = feasible['feed_label'].fillna(feasible['feed_regime'])
    leader_idx = (
        feasible.sort_values('succinate_flux', ascending=False)
        .groupby('feed_regime', sort=False)['succinate_flux']
        .idxmax()
    )
    leaders = feasible.loc[leader_idx].copy()
    leaders = leaders.sort_values('succinate_flux', ascending=False)
    acid_columns = ['succinate_flux'] + [col for col in ORGANIC_ACID_COLUMNS if col != 'succinate_flux']
    ordered_columns = [
        'feed_label', 'temperature_tag', 'glucose_limit', 'oxygen_limit', 'proton_limit',
        'ammonium_limit', 'biomass_flux',
    ] + acid_columns
    leaders = leaders[ordered_columns]
    rename_map = {
        'feed_label': 'Feed label',
        'temperature_tag': 'Temperature',
        'glucose_limit': 'Glucose limit',
        'oxygen_limit': 'Oxygen limit',
        'proton_limit': 'Proton limit',
        'ammonium_limit': 'Ammonium limit',
        'biomass_flux': 'Biomass flux',
    }
    rename_map.update({col: ORGANIC_ACID_LABELS.get(col, col) for col in acid_columns})
    return leaders.rename(columns=rename_map).reset_index(drop=True)

succinate_context = summarize_succinate_context(results)
succinate_context

### Best-performing conditions per metabolite

The table below reports the environmental settings that maximize the secretion of each tracked acid across the grid search. Use it to cross-check how feed supplements, gas limits, and temperature influence metabolites beyond succinate.

In [ ]:
def summarize_acid_optima(results: pd.DataFrame) -> pd.DataFrame:
    feasible = results[results['status'] == 'optimal'].copy()
    if feasible.empty:
        columns = [
            'metabolite',
            'max_flux',
            'biomass_flux',
            'glucose_limit',
            'oxygen_limit',
            'proton_limit',
            'temperature_tag',
            'feed_label',
        ]
        return pd.DataFrame(columns=columns)
    records = []
    for flux_column in ORGANIC_ACID_COLUMNS:
        label = ORGANIC_ACID_LABELS.get(flux_column, flux_column)
        if feasible[flux_column].notna().any():
            best_idx = feasible[flux_column].idxmax()
            best_row = feasible.loc[best_idx]
            records.append({
                'metabolite': label,
                'max_flux': best_row[flux_column],
                'biomass_flux': best_row['biomass_flux'],
                'glucose_limit': best_row['glucose_limit'],
                'oxygen_limit': best_row['oxygen_limit'],
                'proton_limit': best_row['proton_limit'],
                'temperature_tag': best_row['temperature_tag'],
                'feed_label': best_row['feed_label'],
            })
    summary = pd.DataFrame.from_records(records)
    return summary.sort_values('max_flux', ascending=False).reset_index(drop=True)


acid_optima = summarize_acid_optima(results)
acid_optima

### Interpreting infeasible scenarios
The grid search deliberately explores combinations with very tight oxygen and proton
uptake bounds. Those constraints can prevent the model from satisfying the
non-growth-associated maintenance demand (`r_4041`), especially at 37°C where the
ATP requirement is increased. In those cases the linear program has no feasible
solution, which is why COBRApy reports `Solver status is 'infeasible'`.

Use the following cell to inspect the specific environmental settings that produced
an infeasible status. Relax either the oxygen uptake, the proton uptake, or the
maintenance requirement to recover feasibility.


In [ ]:
infeasible = results[results['status'] != 'optimal'].copy()
infeasible[['glucose_limit', 'oxygen_limit', 'proton_limit', 'temperature_tag', 'feed_label', 'status']].head(10)



## 5. Inspect individual designs

Filter the table to retain feasible solutions (`status == 'optimal'`) and examine
the trade-off between succinate secretion and biomass growth to prioritize
conditions that remain physiologically realistic.


In [ ]:
optimal_results = results[results['status'] == 'optimal'].copy()
optimal_results = optimal_results.sort_values(['succinate_flux', 'biomass_flux'], ascending=[False, False])
optimal_results[['succinate_flux', 'biomass_flux', 'glucose_limit', 'oxygen_limit', 'proton_limit', 'temperature_tag', 'feed_label']].head(10)



## 6. Extract flux distributions for promising scenarios

Use parsimonious FBA (pFBA) to compute flux distributions for specific scenarios.
This helps identify key pathways responsible for succinate overproduction under
the selected environmental settings.


In [ ]:
def sample_flux_distribution(row: pd.Series, model: cobra.Model) -> pd.Series:
    with model:
        apply_kennedy_medium(model, glucose_limit=row['glucose_limit'])
        model.reactions.get_by_id('r_1992').lower_bound = row['oxygen_limit']
        model.reactions.get_by_id('r_1832').lower_bound = row['proton_limit']
        model.reactions.get_by_id('r_1672').lower_bound = row['ammonium_limit']
        if row['temperature_tag'] == '37C':
            model.reactions.get_by_id('r_4041').lower_bound = 10.0
        elif row['temperature_tag'] == '20C':
            model.reactions.get_by_id('r_4041').lower_bound = 6.0
        model.objective = model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        pfba_solution = pfba(model)
    return pfba_solution.fluxes

top_design = optimal_results.iloc[0]
top_fluxes = sample_flux_distribution(top_design, analysis_model)
top_fluxes[top_fluxes.abs() > 1e-3].sort_values(ascending=False).head(20)


## 7. Map reactions influencing succinate production

Succinate secretion depends on the neighbouring reactions that connect the tricarboxylic acid (TCA) cycle, the glyoxylate shunt, and related anaplerotic routes. The helpers below trace the stoichiometric neighbourhood around succinate and report which reactions carry flux in the top succinate-producing scenario.


In [ ]:
SUCCINATE_METABOLITE_IDS = ['s_1458', 's_1459', 's_1460']
KEY_SUCCINATE_SUBSYSTEMS = {
    'Citrate cycle (TCA cycle)',
    'Glyoxylate and dicarboxylate metabolism',
    'Methylglyoxal metabolism',
}

def collect_succinate_neighborhood(
    model: cobra.Model,
    fluxes: pd.Series | None = None,
    depth: int = 2,
    min_flux: float = 1e-6,
    include_subsystems: Iterable[str] | None = None,
) -> pd.DataFrame:
    succinate_ids = list(SUCCINATE_METABOLITE_IDS)
    visited_mets = set(succinate_ids)
    visited_rxns: set[str] = set()
    frontier = list(succinate_ids)
    for _ in range(depth):
        next_frontier: list[str] = []
        for met_id in frontier:
            met = model.metabolites.get_by_id(met_id)
            for rxn in met.reactions:
                visited_rxns.add(rxn.id)
                for neighbor in rxn.metabolites:
                    if neighbor.id not in visited_mets:
                        visited_mets.add(neighbor.id)
                        next_frontier.append(neighbor.id)
        frontier = next_frontier
    if include_subsystems:
        for rxn in model.reactions:
            if rxn.subsystem in include_subsystems:
                visited_rxns.add(rxn.id)
    records: list[dict[str, object]] = []
    for rxn_id in visited_rxns:
        rxn = model.reactions.get_by_id(rxn_id)
        flux_value = None
        abs_flux = None
        if fluxes is not None and rxn_id in fluxes.index:
            flux_value = float(fluxes[rxn_id])
            abs_flux = abs(flux_value)
            if abs_flux < min_flux:
                continue
        stoich = sum(coeff for met, coeff in rxn.metabolites.items() if met.id in succinate_ids)
        records.append({
            'reaction_id': rxn.id,
            'name': rxn.name,
            'subsystem': rxn.subsystem or 'Unassigned',
            'succinate_stoichiometry': stoich,
            'flux': flux_value,
            'abs_flux': abs_flux,
        })
    table = pd.DataFrame(records)
    if fluxes is not None and not table.empty:
        table = table.sort_values('abs_flux', ascending=False)
    else:
        table = table.sort_values('reaction_id')
    return table.reset_index(drop=True)

def summarize_succinate_subsystems(table: pd.DataFrame) -> pd.DataFrame:
    counts = table.groupby('subsystem')['reaction_id'].count().sort_values(ascending=False)
    return counts.rename('reaction_count').reset_index()

def plot_succinate_network(
    model: cobra.Model,
    table: pd.DataFrame,
    fluxes: pd.Series | None = None,
    max_reactions: int = 20,
) -> None:
    if table.empty:
        raise ValueError('No reactions available to visualise.')
    selected = table.head(max_reactions)
    if fluxes is not None and 'abs_flux' in table.columns:
        selected = table.nlargest(max_reactions, 'abs_flux')
    reaction_ids = selected['reaction_id'].tolist()
    graph = nx.DiGraph()
    for rxn_id in reaction_ids:
        rxn = model.reactions.get_by_id(rxn_id)
        graph.add_node(
            rxn_id,
            kind='reaction',
            label=rxn.id,
            flux=float(fluxes[rxn_id]) if fluxes is not None and rxn_id in fluxes.index else 0.0,
        )
        for met, coeff in rxn.metabolites.items():
            label = f"{met.name} [{met.compartment}]"
            if met.id in SUCCINATE_METABOLITE_IDS:
                label = f"Succinate [{met.compartment}]"
            graph.add_node(met.id, kind='metabolite', label=label)
            if coeff < 0:
                graph.add_edge(met.id, rxn_id, weight=abs(coeff))
            else:
                graph.add_edge(rxn_id, met.id, weight=abs(coeff))
    pos = nx.spring_layout(graph, seed=42)
    reaction_nodes = [n for n, attrs in graph.nodes(data=True) if attrs.get('kind') == 'reaction']
    metabolite_nodes = [n for n, attrs in graph.nodes(data=True) if attrs.get('kind') == 'metabolite']
    flux_values = [abs(graph.nodes[n].get('flux', 0.0)) for n in reaction_nodes]
    cmap = plt.cm.viridis
    if flux_values and max(flux_values) > 0:
        reaction_colors = [cmap(val / max(flux_values)) for val in flux_values]
    else:
        reaction_colors = [cmap(0.0) for _ in reaction_nodes]
    metabolite_colors = ['#f4b942' if n in SUCCINATE_METABOLITE_IDS else '#6baed6' for n in metabolite_nodes]
    fig, ax = plt.subplots(figsize=(12, 8))
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=metabolite_nodes,
        node_color=metabolite_colors,
        node_shape='o',
        node_size=900,
        ax=ax,
    )
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=reaction_nodes,
        node_color=reaction_colors,
        node_shape='s',
        node_size=700,
        ax=ax,
    )
    nx.draw_networkx_edges(graph, pos, arrowstyle='-|>', arrowsize=12, edge_color='#555555', width=1.5, ax=ax)
    labels = {node: attrs['label'] for node, attrs in graph.nodes(data=True)}
    nx.draw_networkx_labels(graph, pos, labels=labels, font_size=8, ax=ax)
    ax.set_title('Succinate-centred reaction network (top flux routes)')
    ax.axis('off')
    if flux_values and max(flux_values) > 0:
        norm = plt.Normalize(vmin=0, vmax=max(flux_values))
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('Relative pFBA flux (|mmol gDW⁻¹ h⁻¹|)')
    plt.show()


In [ ]:
succinate_neighborhood = collect_succinate_neighborhood(
    analysis_model,
    fluxes=top_fluxes,
    depth=2,
    include_subsystems=KEY_SUCCINATE_SUBSYSTEMS,
)
succinate_neighborhood[['reaction_id', 'name', 'subsystem', 'flux']].head(20)


In [ ]:
summarize_succinate_subsystems(succinate_neighborhood)


### Visualise the succinate neighbourhood

The network plot highlights the metabolites (circles) and reactions (squares) that channel flux towards succinate under the selected conditions. Node colours for reactions follow the relative pFBA flux. Reduce `max_reactions` if the visualisation becomes crowded.


In [ ]:
plot_succinate_network(analysis_model, succinate_neighborhood, fluxes=top_fluxes, max_reactions=18)


## 8. Next steps

* Constrain succinate secretion to a fixed minimum and re-maximize biomass to
  benchmark trade-offs between growth and production.
* Introduce additional feeds (e.g., glycerol or acetate) by adjusting the
  corresponding exchange lower bounds before solving the optimization.
* Export the most promising flux distributions and perform pathway enrichment or
  shadow price analysis to understand metabolic bottlenecks.
